# Extracting rows with salary information

Emilio Lehoucq - 8/20/24

## Importing libraries

In [1]:
import pandas as pd
import re

## Reading data

In [2]:
df = pd.read_csv('cesnet_data_august_20_2024.csv')
print(f'Number of rows: {len(df)}')

Number of rows: 4092


## Identifying rows with salary information

In [3]:
# Columns to store data
df['salary_potential_string'] = None
df['salary_potential_flag'] = None

# Number of characters around the potential salary to extract
N_CHARACTERS = 60

# Loop through each row in the dataframe
for i, row in df.iterrows():
    # Text of the message
    text = row['text_message']
    # Check if there seems to be a money sign followed by digits
    salary_flag = re.finditer(r'[€£$]\s?\d{1,6}', text)
    # Variables to store info
    list_salary_potential_strings = []
    list_salary_potential_flags = []
    # Iterate over potential matches
    for match in salary_flag:
        # Get the index where the info seems to start
        start_idx = match.start()
        # Get the index where the info seems to end
        end_idx = match.end()
        # Get characters around the info
        start_string = start_idx - N_CHARACTERS
        end_string = end_idx + N_CHARACTERS
        # Extract info
        salary_potential_string = text[start_string:end_string].replace('\n', ' ').strip().lower()
        # Check that the string includes the word 'salary' or 'compensation' or 'pay'
        if 'salary' in salary_potential_string or 'compensation' in salary_potential_string or 'pay' in salary_potential_string:
            # Store info
            list_salary_potential_strings.append(salary_potential_string)
            list_salary_potential_flags.append(True)
    # Store info in df
    df.at[i, 'salary_potential_flag'] = any(list_salary_potential_flags)
    if len(list_salary_potential_strings) > 0:
        df.at[i, 'salary_potential_string'] = list_salary_potential_strings
    # If no salary found so far
    else:
        # Store None in df
        df.at[i, 'salary_potential_string'] = None
        # # I WROTE THE CODE BELOW TO CHECK FOR SALARY INFO IN CASE THERE WAS NO MONEY SIGN
        # # BUT I LOOKED AT THE OUTPUT AND ONLY THREE OF THOSE ROWS HAD SALARIES, SO I'M COMMENTING THIS OUT AND ADDING THE ROWS MANUALLY
        # # THE THREE ROWS ARE: 1144, 2365, 2366
        # # Check if text mentions keywords
        # if 'salary' in text or 'compensation' in text or 'pay' in text:
        #     # Check if text mentions numbers that look like salary
        #     salary_flag = re.finditer(r'\b\d{1,3}[,.]?\d{3}\b', text)
        #     # Iterate over potential matches
        #     list_salary_potential_strings = []
        #     list_salary_potential_flags = []
        #     for match in salary_flag:
        #         # Get the index where the info seems to start
        #         start_idx = match.start()
        #         # Get the index where the info seems to end
        #         end_idx = match.end()
        #         # Get characters around the info
        #         start_string = start_idx - N_CHARACTERS
        #         end_string = end_idx + N_CHARACTERS
        #         # Extract info
        #         salary_potential_string = text[start_string:end_string].replace('\n', ' ').strip().lower()
        #         # Check that the string includes the word 'salary' or 'compensation' or 'pay'
        #         if 'salary' in salary_potential_string or 'compensation' in salary_potential_string or 'pay' in salary_potential_string:
        #             # Store info
        #             list_salary_potential_strings.append(salary_potential_string)
        #             list_salary_potential_flags.append(True)
        #     # If there seems to be salary info
        #     if any(list_salary_potential_flags):
        #         # Store info in df
        #         df.at[i, 'salary_potential_flag'] = True
        #         df.at[i, 'salary_potential_string'] = list_salary_potential_strings
        #         # Iterate over potential matches
        #         print("====================================================================")
        #         print(f"Row {i}")
        #         for potential_match in list_salary_potential_strings:
        #             print(potential_match)

# Get subset of rows with potential salary
df_subset = pd.concat([df[df['salary_potential_string'].notna()], df.loc[[1144, 2365, 2366]]])
print(f"There are {df_subset.shape[0]} rows with potential salary.")

There are 129 rows with potential salary.


Please note that rows 1144, 2365, 2366 are missing on salary_potential_string and have False on salary_potential_flag.

This is what the resulting data looks like:

In [4]:
df_subset

,week,url_compilation,url_posting,timestamp,source_code_message,text_message,salary_potential_string,salary_potential_flag
2,"April 2023, Week 2",https://listserv.kent.edu/cgi-bin/wa.exe?A1=in...,https://listserv.kent.edu/cgi-bin/wa.exe?A2=in...,2024-08-16 17:10:44,<html><head>\n<title>LISTSERV 16.5 - CESNET-L ...,LISTSERV 16.5 - CESNET-L Archives\n Print\n As...,[r: salary range: associate professor: general...,True
27,"July 2020, Week 4",https://listserv.kent.edu/cgi-bin/wa.exe?A1=in...,https://listserv.kent.edu/cgi-bin/wa.exe?A2=in...,2024-08-19 12:20:15,<html><head>\n<title>LISTSERV 16.5 - CESNET-L ...,LISTSERV 16.5 - CESNET-L Archives\n Print\n Th...,"[ng instructions about the option you choose, ...",True
57,"February 2017, Week 4",https://listserv.kent.edu/cgi-bin/wa.exe?A1=in...,https://listserv.kent.edu/cgi-bin/wa.exe?A2=in...,2024-08-20 08:25:35,<html><head>\n<title>LISTSERV 16.5 - CESNET-L ...,LISTSERV 16.5 - CESNET-L Archives\n Print\n De...,[ls within the following salary ranges: assist...,True
94,"October 2017, Week 2",https://listserv.kent.edu/cgi-bin/wa.exe?A1=in...,https://listserv.kent.edu/cgi-bin/wa.exe?A2=in...,2024-08-20 07:56:59,<html><head>\n<title>LISTSERV 16.5 - CESNET-L ...,LISTSERV 16.5 - CESNET-L Archives\n Print\n We...,[nclusion. salary & benefits: the minimum star...,True
120,"May 2024, Week 3",https://listserv.kent.edu/cgi-bin/wa.exe?A1=in...,https://listserv.kent.edu/cgi-bin/wa.exe?A2=in...,2024-08-16 15:44:29,<html><head>\n<title>LISTSERV 16.5 - CESNET-L ...,LISTSERV 16.5 - CESNET-L Archives\n Print\n As...,[he united states on a full-time basis.” tenur...,True
...,...,...,...,...,...,...,...,...
4000,"June 2024, Week 4",https://listserv.kent.edu/cgi-bin/wa.exe?A1=in...,https://listserv.kent.edu/cgi-bin/wa.exe?A2=in...,2024-08-16 15:44:25,<html><head>\n<title>LISTSERV 16.5 - CESNET-L ...,LISTSERV 16.5 - CESNET-L Archives\n Print\n Ce...,"[ing. we're offering a salary, depending on ex...",True
4059,"January 2019, Week 3",https://listserv.kent.edu/cgi-bin/wa.exe?A1=in...,https://listserv.kent.edu/cgi-bin/wa.exe?A2=in...,2024-08-19 13:55:58,<html><head>\n<title>LISTSERV 16.5 - CESNET-L ...,LISTSERV 16.5 - CESNET-L Archives\n Print\n He...,"[th setting, a public/private school setting, ...",True
1144,"December 2019, Week 2",https://listserv.kent.edu/cgi-bin/wa.exe?A1=in...,https://listserv.kent.edu/cgi-bin/wa.exe?A2=in...,2024-08-19 12:58:14,<html><head>\n<title>LISTSERV 16.5 - CESNET-L ...,LISTSERV 16.5 - CESNET-L Archives\n Print\n Ch...,None,False
2365,"December 2019, Week 1",https://listserv.kent.edu/cgi-bin/wa.exe?A1=in...,https://listserv.kent.edu/cgi-bin/wa.exe?A2=in...,2024-08-19 12:55:53,<html><head>\n<title>LISTSERV 16.5 - CESNET-L ...,LISTSERV 16.5 - CESNET-L Archives\n Print\n De...,None,False


## Save to CSV

In [5]:
df_subset.to_csv('cesnet_data_august_20_2024_subset_with_salary.csv', index=False)